In [180]:
%load_ext autoreload
%aimport data_creater
%aimport model
%autoreload 1

from data_creater import *
from model import *
import re, copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:
stocks = companies()
symbols = stocks['Symbol'].values.tolist()

In [97]:
stocks = companies()
symbols = stocks['Symbol'].values.tolist()
symbol = 'AAPL'
if os.path.isfile('./data/{0}/important_symbol.json'.format(symbol)):
    with open('./data/{0}/important_symbol.json'.format(symbol), 'r') as json_file:
        loaded_model_json = json_file.read()
        loaded_model_json = json.loads(loaded_model_json)
        symbols = list(loaded_model_json['score'].keys())

other_symbols = [symbol_tem for symbol_tem in symbols if symbol_tem != symbol]
df = pd.read_csv('./data/{0}/all_normalized.csv'.format(symbol), index_col=[0], parse_dates=[0])
symbol_columns = [symbol_tem for symbol_tem in list(df.columns.values) if bool(re.match('normal.*', symbol_tem))]

box = []
for symbol_tem in other_symbols:
    box_tem = []
    for col in list(df.columns.values):
        if bool(re.match(symbol_tem + '_normal_.*', col)):
            box_tem.append(col)
    box.append(box_tem)

all_combination = []
all_combination.append(symbol_columns)
for box_tem in box:
    all_combination.append(symbol_columns + box_tem)
    
len(all_combination)

31

In [154]:
ticker = 'AAPL'
window_size = 3
seq_obj = MultiSequence(ticker,window_size,1,'g000',all_combination[0])


# sys.exit()
X_train,y_train,X_test,y_test = split_data(seq_obj)
print(seq_obj.X[2])
print(seq_obj.y[2])

seq_obj = MultiSequence(ticker,window_size,1,'s001',all_combination[0])


# sys.exit()
X_train,y_train,X_test,y_test = split_data(seq_obj)
print(seq_obj.X[2])
print(seq_obj.y[2])




[-0.03352107 -0.17754371 -0.3861563   0.70570341  0.54774041  0.66382023
  0.64942526 -0.68993871  3.         -0.26815705 -0.2792827  -0.40817555
  0.09049326  0.47626233  0.44672878  0.12399471 -0.64910956  4.
 -0.60335345 -0.33233866 -0.42354147  0.12243274 -0.06654407  0.41229764
  0.15052712 -0.64951155  5.        ]
[1]
[-0.03352107 -0.17754371 -0.3861563   0.70570341  0.54774041  0.66382023
  0.64942526 -0.68993871  3.         -0.26815705 -0.2792827  -0.40817555
  0.09049326  0.47626233  0.44672878  0.12399471 -0.64910956  4.
 -0.60335345 -0.33233866 -0.42354147  0.12243274 -0.06654407  0.41229764
  0.15052712 -0.64951155  5.        ]
[0]


In [171]:
window_sizes = [3,4,5]

In [210]:
result = model_selector('AAPL', window_sizes, target_length=1,target_theme='s001',verbose=1,column=all_combination[2])

print("\nResults : ")
print("-"*60)
print(result[0])

print(result[1])
ModelLoader.save(result[1]['ticker'],result[0],result[1],force_overwrite=False)


*** Best Model Selection for AAPL ***

Window size: 3
------------------------------------------------------------
    y_test_true  y_test_pred
0             1            0
1             0            0
2             0            0
3             0            1
4             0            1
5             0            0
6             0            0
7             0            1
8             0            1
9             0            1
10            0            1
11            0            0
12            0            1
13            0            1
14            0            1
15            1            1
16            0            0
17            0            1
18            0            1
19            0            1
20            0            1
21            0            0
22            0            1
23            0            1
24            0            1
25            0            1
26            0            0
27            0            1
28            0            1
[0.88888889 0.0

In [211]:
for i in range(len(all_combination)):
    result = model_selector('AAPL', window_sizes, target_length=1,target_theme='s001',verbose=1,column=all_combination[i])

    print("\nResults : ")
    print("-"*60)
    print(result[0])

    print(result[1])
    ModelLoader.save(result[1]['ticker'],result[0],result[1],force_overwrite=False)

*** Best Model Selection for AAPL ***

Window size: 3
------------------------------------------------------------
    y_test_true  y_test_pred
0             1            0
1             0            0
2             0            0
3             0            1
4             0            1
5             0            0
6             0            0
7             0            0
8             0            1
9             0            1
10            0            0
11            0            0
12            0            0
13            0            1
14            0            1
15            1            0
16            0            0
17            0            1
18            0            1
19            0            1
20            0            0
21            0            0
22            0            0
23            0            1
24            0            1
25            0            0
26            0            0
27            0            0
28            0            1
[0.88235294 0. 

    y_test_true  y_test_pred
0             1            0
1             0            0
2             0            0
3             0            1
4             0            1
5             0            0
6             0            0
7             0            1
8             0            1
9             0            1
10            0            1
11            0            0
12            0            1
13            0            1
14            0            1
15            1            1
16            0            0
17            0            1
18            0            1
19            0            1
20            0            1
21            0            0
22            0            1
23            0            1
24            0            1
25            0            1
26            0            0
27            0            1
28            0            1
[0.88888889 0.05      ]
1    > Window size: 3 Score: 0.5794training_F1: 0.7388 training_precision: 0.7143  training_recall: 0.7143

    y_test_true  y_test_pred
0             1            0
1             0            0
2             0            1
3             0            1
4             0            1
5             0            0
6             0            0
7             0            1
8             0            1
9             0            1
10            0            0
11            0            0
12            0            0
13            0            1
14            0            1
15            1            0
16            0            0
17            0            1
18            0            1
19            0            1
20            0            1
21            0            0
22            0            1
23            0            1
24            0            1
25            0            1
26            0            0
27            0            1
28            0            1
[0.81818182 0.        ]
1    > Window size: 3 Score: 0.5661training_F1: 0.6911 training_precision: 0.6607  training_recall: 0.6607

AttributeError: 'MultiSequence' object has no attribute '_SequenceBase__data_normal'

In [148]:
dropout =  0.25 ; rate = learn_rate = 0.001 ; epoch = 30
seq_obj = MultiSequence('AAPL',4,1,'s001', all_combination[1])
X_train,y_train,X_test,y_test = split_data(seq_obj)
y_train = list(chain(*y_train.tolist()))
y_test = list(chain(*y_test.tolist()))

from sklearn.svm import SVC
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train,y_train)
svm.predict(X_test)

# model = Sequential()
# model.add(Bidirectional(LSTM(X_train.shape[1],return_sequences=False),input_shape=(X_train.shape[1:])))
# model.add(Dense(X_train.shape[1]))
# model.add(Dropout(dropout))
# model.add(Dense(y_train.shape[1],activation='tanh'))
# optimizer = RMSprop(lr=learn_rate)
# model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
# model = final_model(X_train,y_train,rate,dropout)
# model.fit(X_train,y_train,epochs=epoch,batch_size=batch_size)

# raining_loss = model.evaluate(X_train,y_train,verbose=0)[0]
# training_acc = model.evaluate(X_train,y_train,verbose=0)[1]
# testing_loss = model.evaluate(X_test,y_test,verbose=0)[0]
# testing_acc = model.evaluate(X_test,y_test,verbose=0)[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [131]:
symbol = 'AAPL' ; __target_length = 1 ; __target_theme = 's001'

__best_model = ModelLoader(symbol, __target_length, __target_theme)

__prop_path = "./model/{0}/{1}_{2}_train_props.json"
with open(__prop_path.format(symbol, symbol+str(__target_length), __target_theme), 'r') as json_file:
    loaded_model_json = json_file.read()
    loaded_model_json = json.loads(loaded_model_json)
    colname = loaded_model_json['colname']

seq_obj = MultiSequence(symbol, __best_model.window_size,__target_length,__target_theme,column=colname)
orig_prices = seq_obj.origin_close
orig_date = seq_obj.all_date
ans = [ '{0:.3f}%'.format(100*round(value, 4)) for value in seq_obj.rate]   
orig_price_df = pd.DataFrame.from_dict({'date': orig_date, 'close_price': orig_prices, 'ans': ans})
orig_price_df.index = range(len(orig_price_df))

predict_prob = __best_model.model.predict(seq_obj.Xpred)
pred_prob_df = pd.DataFrame(predict_prob, columns = [symbol+str(__target_length) + '_' + __target_theme])

pred_prob_df.index = range(__target_length + __best_model.window_size - 1, len(pred_prob_df) + __target_length + __best_model.window_size - 1)  
df = orig_price_df.merge(pred_prob_df, how = 'outer', left_index=True, right_index=True)
display(df)

./model/AAPL/AAPL1_s001_model.json


,date,close_price,ans,AAPL1_s001
0,2018-11-26,172.035172,1.350%,NaN
1,2018-11-27,171.660797,-0.220%,NaN
2,2018-11-28,178.261627,3.850%,NaN
3,2018-11-29,176.892197,-0.770%,0.123472
4,2018-11-30,175.936554,-0.540%,0.292089
...,...,...,...,...
241,2019-11-11,262.200012,0.790%,0.419312
242,2019-11-12,261.959991,-0.090%,0.208137
243,2019-11-13,264.470001,0.960%,0.166552
244,2019-11-14,262.640015,-0.690%,0.198162


In [130]:
df.tail(60)

,date,close_price,ans,AAPL1_s001
186,2019-08-23,202.033447,-4.620%,0.238106
187,2019-08-26,205.871918,1.900%,0.436899
188,2019-08-27,203.548889,-1.130%,0.190585
189,2019-08-28,204.914795,0.670%,0.228366
190,2019-08-29,208.384369,1.690%,0.068299
191,2019-08-30,208.115189,-0.130%,0.197377
192,2019-09-03,205.084274,-1.460%,0.330177
193,2019-09-04,208.563843,1.700%,0.223813
194,2019-09-05,212.641586,1.960%,0.160933
195,2019-09-06,212.621643,-0.010%,0.192618


In [72]:
import os, re
symbol = 'AAPL'
path = './model_step/{0}'.format(symbol)
files = []
for r, d, f in os.walk(path):
    for file in f:
        if 'props' in file:
            files.append(os.path.join(r, file))

df = pd.DataFrame(columns=['other_symbol', 'score'])
for i in range(len(files)):
    try:
        with open(files[i], 'r') as json_file:
            loaded_model_json = json_file.read()
            loaded_model_json = json.loads(loaded_model_json)

        other_symbol_name = '.'.join(list((set([col.split('_')[0] for col in loaded_model_json['colname'] if not re.match('normal.*', col)]))))
        if other_symbol_name == '':
            other_symbol_name = symbol
        score = loaded_model_json['score']
        df.loc[i] = [other_symbol_name, score]
        
        if score > 0.7:
            print(files[i])
        
    except:
        continue
df = df.sort_values(by = 'score', ascending = False).head(30)
df.index = df['other_symbol']
df = df.drop(columns = ['other_symbol'])
df_json = df.to_dict()
print(df_json)
with open('./data/{0}/important_symbol.json'.format(symbol), 'w', encoding='utf-8') as f:
    json.dump(df_json, f, ensure_ascii=False, indent=4)

{'score': {'VIAB': 0.6628, 'FRC': 0.6616, 'PPL': 0.653, 'LOW': 0.6452, 'CAG': 0.6438, 'MKTX': 0.6429, 'ALGN': 0.6427, 'DLR': 0.6422, 'NRG': 0.6407, 'DFS': 0.6398, 'ABT': 0.6366, 'MRO': 0.6363, 'ROL': 0.6362, 'DISCK': 0.6352, 'MDLZ': 0.6342, 'CME': 0.6327, 'AVY': 0.6327, 'AMD': 0.6325, 'DXC': 0.6323, 'HRB': 0.6319, 'MOS': 0.6317, 'FANG': 0.6314, 'CERN': 0.6311, 'INCY': 0.6308, 'UPS': 0.6288, 'PNR': 0.6276, 'AXP': 0.6267, 'TFX': 0.6267, 'HSIC': 0.6262, 'FAST': 0.6252}}
